In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres. 
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-02-22 15:33:44--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.2’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2022-02-22 15:33:44 (6.57 MB/s) - ‘postgresql-42.2.16.jar.2’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge-D2").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Load Amazon Data into Spark DataFrame
from pyspark import SparkFiles
url = "https://mshideler-bigdatabucket.s3.amazonaws.com/amazon_reviews_us_Pet_Products_v1_00.tsv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...| 2015-08-31|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...|    Pet Products|          2|    

In [5]:
# Create the vine DataFrame
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| REAKC26P07MDN|          5|            0|          0|   N|                Y|
|R3NU7OMZ4HQIEG|          2|            0|          1|   N|                Y|
|R14QJW3XF8QO1P|          5|            0|          0|   N|                Y|
|R2HB7AX0394ZGY|          5|            0|          0|   N|                Y|
| RGKMPDQGSAHR3|          5|            0|          0|   N|                Y|
|R1DJCVPQGCV66E|          5|            0|          0|   N|                Y|
|R3V52EAWLPBFQG|          3|            0|          0|   N|                Y|
|R3DKO8J1J28QBI|          2|            0|          0|   N|                Y|
| R764DBXGRNECG|          5|            1|          1|   N|                N|
| RW1853GAT0Z9F|          5|            0|          0|   N|     

In [6]:
# Filter where number of votes is greater than or equal to 20
filtered_vine_df = vine_df.filter('total_votes>=20')

In [7]:
# Filter where helpful_votes / total_votes is greater than or equal to 50%
ratio_vine_df = filtered_vine_df.withColumn('vote_ratio', filtered_vine_df['helpful_votes']/filtered_vine_df['total_votes'])
vine_50perc_df = ratio_vine_df.filter('vote_ratio >= 0.5').select('review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase')

# Determine the total number of votes where helpful_votes / total_votes is greater than or equal to 50%
total_vine_count = vine_50perc_df.count()
total_vine_count

38010

In [8]:
# Filter where the reviews were written under the Vine program
y_vine_df = vine_50perc_df.filter(vine_50perc_df["vine"] == "Y")

# Determine the number of reviews written under the Vine program
y_vine_count = y_vine_df.count()
y_vine_count

170

In [9]:
# Filter where the reviews weren't written under the Vine program
n_vine_df = vine_50perc_df.filter(vine_50perc_df["vine"] == "N")

# Determine the number of reviews not written under the Vine program
n_vine_count = n_vine_df.count()
n_vine_count

37840

In [10]:
# Determine the total number of 5-star reviews
reviews_5stars_df = vine_50perc_df.filter('star_rating == 5')
reviews_5stars_count = reviews_5stars_df.count()
reviews_5stars_count

20677

In [11]:
# Determine the percentage of 5-star reviews for Vine reviews (paid)
reviews_5stars_paid_count = y_vine_df.filter('star_rating == 5').count()
#reviews_5stars_paid_count
percent_5star_paid = reviews_5stars_paid_count / y_vine_count * 100
percent_5star_paid

38.23529411764706

In [12]:
# Determine the percentage of 5-star reviews for regular reviews (unpaid)
reviews_5stars_unpaid_count = n_vine_df.filter('star_rating == 5').count()
#reviews_5stars_unpaid_count
percent_5star_unpaid = reviews_5stars_unpaid_count / n_vine_count * 100
percent_5star_unpaid

54.471458773784356

In [13]:
# Result summary from determining the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).
print(f'The total number of reviews relevant to our analysis equals {total_vine_count:,}.')
print(f'The total number of 5-star reviews equals {reviews_5stars_count:,}.')
print(f'The number of 5-star reviews done by Vine members equals {reviews_5stars_paid_count:,}.')
print(f'The number of 5-star reviews done by non-Vine members equals {reviews_5stars_unpaid_count:,}.')
print(f'The percentage of Vine reviews that had a 5-star rating equals {percent_5star_paid:.1f}%.')
print(f'The percentage of reviews not under the Vine program and that had a 5-star rating equals {percent_5star_unpaid:.1f}%.')

The total number of reviews relevant to our analysis equals 38,010.
The total number of 5-star reviews equals 20,677.
The number of 5-star reviews done by Vine members equals 65.
The number of 5-star reviews done by non-Vine members equals 20,612.
The percentage of Vine reviews that had a 5-star rating equals 38.2%.
The percentage of reviews not under the Vine program and that had a 5-star rating equals 54.5%.
